* Flask for REST API Development
* Setup Flask Project for REST APIs
* Develop GET API for Users
* Exercise and Solution - Develop GET API for Courses

* Flask for REST API Development

Flask is more popular for REST API Development. In those scenarios, the front end will be developed using Java Scrip frameworks such as Angular, React, Vue, etc.

* Setup Flask Project for REST APIs

1. Setup Project using VS Code (`sales-app-rest`).
2. Create Python Virtual Environment by name `sar-venv`. Make sure it is integrated with the workspace.
3. Create `requirements.txt` with required dependencies and install the dependencies.

```shell
Flask==2.3.2
```

4. Create `app.py` with the following code.

```python
from flask import Flask


app = Flask(__name__)


@app.route('/')
def hello_world():
    return {'message': 'Hello World'}
```

5. Run the application and then validate GET API for base url using below `curl` command.`

```shell
python -m flask run --debug
curl http://localhost:5000 # Run this in another terminal
```

* Develop GET API for Users

1. Add required dependencies for database connectivity.

```shell
Flask==2.3.2
Flask-SQLAlchemy==3.0.3
psycopg2-binary==2.9.6
python-dotenv==1.0.0
```

2. Setup `.env` file with database connectivity information.

```text
SALES_DB_HOST=localhost
SALES_DB_PORT=5432 # in my case it is 5433
SALES_DB_NAME=sales_db
SALES_DB_USER=sales_user
SALES_DB_PASS=itversity
```

3. Add logic `app.py` to use SQLAlchemy for Database Connectivity.

```python
import os
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy


load_dotenv('.env')
db = SQLAlchemy()
app = Flask(__name__)
app.logger.setLevel('INFO')
host = os.environ.get('SALES_DB_HOST')
port = os.environ.get('SALES_DB_PORT')
db_name = os.environ.get('SALES_DB_NAME')
user = os.environ.get('SALES_DB_USER')
password = os.environ.get('SALES_DB_PASS')
app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)


@app.route('/')
def hello_world():
    return 'Hello World'
```

4. Add `models/user.py` with below code.

```python
from app import db


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    first_name = db.Column(db.String)
    last_name = db.Column(db.String)
    username = db.Column(db.String, unique=True, nullable=False)
    email = db.Column(db.String)
```

5. Add `routes/user_routes.py` with below code.

```python
from flask import jsonify, request

from app import app
from app import db
from models.user import User


@app.route('/users')
def users():
    search_email = request.args.get('email', '')  

    if search_email:
        # Query the database and filter users based on the pattern match
        user_recs = db.session.query(User).filter(User.email.like(f'{search_email.lower()}%')).all()
    else:
        # Retrieve all users if no search query is provided
        user_recs = db.session.query(User).all()

    # below commented code throws error as the SQL Alchemy 
    # get response includes attributes such as _sa_instance_state. 
    # The values of these attributes are not JSON Serializable

    # users = list(map(lambda rec: rec.__dict__, user_recs))
    
    # Fix: We need to Serialize Database Model Objects to JSON
    users = []
    for user in user_recs:
        user.__dict__.pop('_sa_instance_state')
        users.append(user.__dict__)

    return jsonify(users)
```

6. Update `app.py` with the code integrating with user routes.

```python
import os
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy


load_dotenv('.env')
db = SQLAlchemy()
app = Flask(__name__)
app.logger.setLevel('INFO')
host = os.environ.get('SALES_DB_HOST')
port = os.environ.get('SALES_DB_PORT')
db_name = os.environ.get('SALES_DB_NAME')
user = os.environ.get('SALES_DB_USER')
password = os.environ.get('SALES_DB_PASS')
app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)

from routes import user_routes


@app.route('/')
def hello_world():
    return 'Hello World'
```

7. Validate the application using Flask Shell

```python
from werkzeug.test import Client
client = Client(app)

client.get('/users').get_json()
```

* Exercise - Develop APIs for Courses

1. Create Model for Course
2. Add Route to list the existing courses
3. Make sure `app.py` is updated to use the route
4. Validate all the APIs using Flask Shell 

* Exercise - Develop APIs for Courses

1. Create Model for Course (`models/course.py`)

```python
from app import db


class Course(db.Model):
    __tablename__ = 'courses'
    course_id = db.Column(db.Integer, primary_key=True)
    course_name = db.Column(db.String)
    course_author = db.Column(db.String)
    course_endpoint = db.Column(db.String)
```

2. Add Route to list the existing courses

```python
from flask import jsonify, request

from app import app
from app import db
from models.course import Course


@app.route('/courses')
def courses():
    search_query = request.args.get('search', '')

    if search_query:
        # Query the database for courses matching the search query
        course_recs = db.session. \
            query(Course). \
            filter(Course.course_name.ilike(f"%{search_query}%")). \
            all()
    else:
        # Query the database for all courses
        course_recs = db.session.query(Course).all()

    courses = []
    for course in course_recs:
        course.__dict__.pop('_sa_instance_state')
        courses.append(course.__dict__)
    return jsonify(courses)
```

3. Make sure `app.py` is updated to use the route

```python
from routes import user_routes, course_routes # Update existing line
```

4. Validate GET API for users using Flask Shell

```python
from werkzeug.test import Client
client = Client(app)

client.get('/courses').get_json()
```